# Try Pylandsat

In [6]:
pip install pylandsat pandas geopandas matplotlib

  Using cached matplotlib-3.10.1-cp311-cp311-win_amd64.whl.metadata (11 kB)
  Using cached contourpy-1.3.1-cp311-cp311-win_amd64.whl.metadata (5.4 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.56.0-cp311-cp311-win_amd64.whl.metadata (103 kB)
  Using cached kiwisolver-1.4.8-cp311-cp311-win_amd64.whl.metadata (6.3 kB)
  Using cached pillow-11.1.0-cp311-cp311-win_amd64.whl.metadata (9.3 kB)
Using cached matplotlib-3.10.1-cp311-cp311-win_amd64.whl (8.1 MB)
Using cached contourpy-1.3.1-cp311-cp311-win_amd64.whl (219 kB)
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
Using cached fonttools-4.56.0-cp311-cp311-win_amd64.whl (2.2 MB)
Using cached kiwisolver-1.4.8-cp311-cp311-win_amd64.whl (71 kB)
Using cached pillow-11.1.0-cp311-cp311-win_amd64.whl (2.6 MB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
from datetime import datetime

from shapely.geometry import Point
from pylandsat import Catalog, Product
import pandas as pd
catalog = Catalog()

begin = datetime(2010, 1, 1)
end = datetime(2020, 1, 1)
geom = Point(4.34, 50.85)

# Results are returned as a list
scenes = catalog.search(
    begin=begin,
    end=end,
    geom=geom,
    sensors=['LE07', 'LC08']
)

# Get the product ID of the first scene
product_id = scenes[0].get("product_id")

# Download the scene
product = Product(product_id)
product.download(out_dir='data')

# The output of catalog.search() can be converted to a DataFrame
# for further processing. For instance:
# Get the product ID of the scene with the lowest cloud cover


df = pd.DataFrame.from_dict(scenes)
df.set_index(["product_id"], inplace=True)
df = df.sort_values(by='cloud_cover', ascending=True)
product_id = df.index[0]

c:\Users\marga\Documents\EPFL\Cours\S2\Design_project\codes\Satellite_glacier\.venv\Lib\site-packages\rasterio\path.py:15: RasterioDeprecationWarning: rasterio.path will be removed in version 1.4.
  warnings.warn(


OperationalError: Le module spécifié est introuvable.


In [4]:
import json
import geopandas as gpd
from pylandsat import PyLandsat

# Load credentials from the configuration file
with open('configLandsat.json') as f:
    config = json.load(f)

username = config['username']
password = config['password']

# Initialize the PyLandsat object
landsat = PyLandsat(username=username, password=password)

# Read the shapefile
shapefile_path = r'C:\Users\marga\Documents\EPFL\Cours\S2\Design_project\Vevey2.shp'
gdf = gpd.read_file(shapefile_path)

# Extract the geometry (assuming a single feature in the shapefile)
geometry = gdf.geometry[0]

# Define the search parameters
start_date = '2023-01-01'
end_date = '2023-12-31'
max_cloud_cover = 10

# Convert the geometry to a bounding box (minx, miny, maxx, maxy)
minx, miny, maxx, maxy = geometry.bounds

# Search for Landsat scenes within the bounding box
scenes = landsat.search_by_coordinates(
    start_date, end_date, minx, miny, maxx, maxy, max_cloud_cover
)

# Download the first scene in the search results
if scenes:
    landsat.download(scenes[0]['entityId'], output_dir='./landsat_data')
else:
    print("No scenes found matching the criteria.")


ImportError: cannot import name 'PyLandsat' from 'pylandsat' (c:\Users\marga\Documents\EPFL\Cours\S2\Design_project\codes\Satellite_glacier\.venv\Lib\site-packages\pylandsat\__init__.py)

In [7]:
import numpy as np
import rasterio
import matplotlib.pyplot as plt
from pylandsat import Scene

# Access data
scene = Scene('data/LE07_L1TP_205050_19991104_20170216_01_T1')
print(scene.available_bands())
print(scene.product_id)
print(scene.sensor)
print(scene.date)

# Access MTL metadata
print(scene.mtl['IMAGE_ATTRIBUTES']['CLOUD_COVER_LAND'])

# Quality band
plt.imshow(scene.quality.read())

# Access band data
nir = scene.nir.read(1)
red = scene.red.read(1)
ndvi = (nir + red) / (nir - red)

# Access band metadata
print(scene.nir.bname)
print(scene.nir.fname)
print(scene.nir.profile)
print(scene.nir.width, scene.nir.height)
print(scene.nir.crs)

# Use reflectance values instead of DN
nir = scene.nir.to_reflectance()

# ..or brightness temperature
tirs = scene.tirs.to_brightness_temperature()

# Save file to disk
with rasterio.open('temperature.tif', 'w', **scene.tirs.profile) as dst:
    dst.write(tirs, 1)

FileNotFoundError: [WinError 3] Le chemin d’accès spécifié est introuvable: 'c:\\Users\\marga\\Documents\\EPFL\\Cours\\S2\\Design_project\\codes\\Satellite_glacier\\draft\\data\\LE07_L1TP_205050_19991104_20170216_01_T1'

# Using Landsatxplore

In [ ]:
pip install landsatxplore json

  Using cached click-7.1.2-py2.py3-none-any.whl.metadata (2.9 kB)
Using cached click-7.1.2-py2.py3-none-any.whl (82 kB)
  Attempting uninstall: click
    Found existing installation: click 8.1.8
    Uninstalling click-8.1.8:
      Successfully uninstalled click-8.1.8
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fiona 1.10.1 requires click~=8.0, but you have click 7.1.2 which is incompatible.


In [ ]:
import json
from landsatxplore.api import API

# Load credentials from the configuration file
with open('configLandsat.json') as f:
    config = json.load(f)

username = config['username']
password = config['password']

# Initialize the API instance
api = API(username, password)


USGSError: NOT_FOUND: Invalid Endpoint.

In [ ]:

# Search for Landsat TM scenes
scenes = api.search(
    dataset='landsat_tm_c2_l1',
    latitude=50.85,
    longitude=-4.35,
    start_date='1995-01-01',
    end_date='1995-10-01',
    max_cloud_cover=10
)

print(f"{len(scenes)} scenes found.")

# Process the result
for scene in scenes:
    print(scene['acquisition_date'].strftime('%Y-%m-%d'))
    # Write scene footprints to disk
    fname = f"{scene['landsat_product_id']}.geojson"
    with open(fname, "w") as f:
        json.dump(scene['spatial_coverage'].__geo_interface__, f)

api.logout()

USGSError: NOT_FOUND: Invalid Endpoint.

In [ ]:
from landsatxplore.earthexplorer import EarthExplorer

ee = EarthExplorer('Margaux_ccd', 'tnEh,H~$E+5W')

ee.download('LT51960471995178MPS00', output_dir='./data')

ee.logout()

# Using